# Import Necessary Libraries

In [1]:
import numpy as np
import os
import mne
import pywt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, SimpleRNN
from tensorflow.keras.models import Sequential
from sklearn.model_selection import GroupKFold
import tensorflow as tf
from tensorflow.keras import layers, models
from scipy.interpolate import interp1d

# Define X and Y

In [2]:
# Load the saved features from the .npz file
data = np.load('task_merged_features.npz') 

# Access the concatenated features from the .npz file
X_merged_features = data['X_merged_features']

# Close the loaded file
data.close()

#print the merged array
X_merged_features

array([[[-1.06148503e-04, -1.06694601e-04, -1.07029483e-04, ...,
          4.28387813e-10,  1.15596322e-01,  5.15763781e+00],
        [-3.84645605e-05, -3.71460922e-05, -3.56534521e-05, ...,
          1.11612094e-10,  2.25178992e-01,  3.13333729e+00],
        [-2.64800307e-05, -2.69086897e-05, -2.82668046e-05, ...,
          1.17426291e-10,  2.13541218e-01,  3.09660245e+00],
        ...,
        [ 5.96396755e-05,  5.91926693e-05,  5.86861593e-05, ...,
          1.18581890e-10,  2.25537207e-01,  2.81157912e+00],
        [-2.79652708e-05, -2.81712055e-05, -2.97970491e-05, ...,
          1.34312102e-10,  2.13075454e-01,  3.19819047e+00],
        [ 1.38124990e-05,  1.54131628e-05,  2.01976292e-05, ...,
          2.71039791e-10,  2.38988451e-01,  2.70227713e+00]],

       [[ 1.07158417e-04,  1.07484494e-04,  1.08244955e-04, ...,
          1.29960133e-10,  2.04071604e-01,  3.39227165e+00],
        [-1.66651973e-05, -1.64979475e-05, -1.75584333e-05, ...,
          9.53881075e-11,  2.56205157e

In [3]:
data = np.load('task_labels.npz')

Y = data['Y']

data.close() 

Y

array([0, 0, 0, ..., 1, 1, 1])

In [4]:
data = np.load('task_groups.npz')

group = data['group']

data.close() 

Y

array([0, 0, 0, ..., 1, 1, 1])

# Reshape Array

In [5]:
Y=Y
groups_array = np.hstack(group)

In [6]:
# Check the shape of the concatenated arrays
print(X_merged_features.shape, Y.shape, groups_array.shape)

(38049, 20, 238) (38049,) (38049,)


In [7]:
# # Move the channel axis to the last dimension in the EEG data array
# epochs_array = np.moveaxis(X_merged_features, 1, 2)

# # Check the shape of the modified EEG data array
# epochs_array.shape

# Develop the Autoregression RNN Model

In [8]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, Dropout, BatchNormalization, Attention
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import numpy as np

# Define the improved CNN-LSTM hybrid model
def cnn_lstm_model(input_shape):

    learning_rate = 0.001
    dropout_rate = 0.3
    # Create a Sequential model
    model = Sequential()
    
    # Add Convolutional layers
    model.add(Conv1D(filters=64, kernel_size=5, activation='relu', padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))
    
    model.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))

    model.add(Conv1D(filters=256, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    # model.add(Dropout(dropout_rate))
    
    # Add Bidirectional LSTM layers
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    # model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    # model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(256)))
    # model.add(Dropout(dropout_rate))

    
    # Add Dense layers
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    optimizer=Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, matthews_corrcoef

# Define the evaluate_model function
def evaluate_model(model, features, labels):
    # Predict labels
    y_pred = model.predict(features)
    y_pred = np.round(y_pred).astype(int)
    
    # Calculate evaluation metrics
    accuracy = accuracy_score(labels, y_pred)
    precision = precision_score(labels, y_pred)
    recall = recall_score(labels, y_pred)
    f1 = f1_score(labels, y_pred)
    roc_auc = roc_auc_score(labels, y_pred)
    
    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(labels, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)
    
    # Calculate kappa and MCC
    total = tp + tn + fp + fn
    observed_accuracy = (tp + tn) / total
    expected_accuracy = ((tp + fp) * (tp + fn) + (tn + fp) * (tn + fn)) / (total ** 2)
    kappa = (observed_accuracy - expected_accuracy) / (1 - expected_accuracy)
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    
    return accuracy, precision, recall, f1, roc_auc, specificity, sensitivity, kappa, mcc

# Train the CNN LSTM Model

In [12]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, CSVLogger
import os
import csv

metrics_list = []

# Initialize confusion matrix sums
total_conf_matrix = [[0, 0], [0, 0]]

# Define the number of folds for cross-validation
n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Create a directory for TensorBoard logs
log_dir = "logs"
os.makedirs(log_dir, exist_ok=True)

# Perform cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(X_merged_features, Y), 1):
    print(f"Fold {fold}")

    # Split the data into training and validation sets
    X_train, X_val = X_merged_features[train_index], X_merged_features[val_index]
    y_train, y_val = Y[train_index], Y[val_index]

    # Standardize features using StandardScaler
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
    X_val_scaled = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)

    # Reshape the data for RNN input (samples, timesteps, features)
    timesteps = X_train_scaled.shape[1]
    features = X_train_scaled.shape[2]
    X_train_rnn = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, features))
    X_val_rnn = X_val_scaled.reshape((X_val_scaled.shape[0], timesteps, features))

    # Create an instance of the autoregression RNN model
    model = cnn_lstm_model((timesteps, features))

    # Set up TensorBoard callback
    tensorboard_callback = TensorBoard(log_dir=os.path.join(log_dir, f'fold_{fold}'))

    # Set up callbacks
    run_name = f"epoch_{fold}"
    checkpoint_filepath = f'{run_name}.keras'
    checkpoint_callback = ModelCheckpoint(
        checkpoint_filepath,
        monitor='val_accuracy',
        save_best_only=True,
        mode='max'
    )

    csv_logger = CSVLogger(f'{run_name}.csv', append=True, separator=';')
    
    # Train the autoregression RNN model
    history = model.fit(X_train_rnn, y_train, epochs=200, batch_size=256, validation_data=(X_val_rnn, y_val), verbose=1, callbacks=[tensorboard_callback, checkpoint_callback, csv_logger])


    # Load the best model before evaluation
    best_model = load_model(checkpoint_filepath)

    # Evaluate client model
    accuracy, precision, recall, f1, sensitivity, specificity, roc_auc, kappa, mcc = evaluate_model(best_model, X_val_rnn, y_val)
    metrics_list.append({
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1': f1,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'ROC_AUC': roc_auc,
        'Kappa': kappa,
        'MCC': mcc
    })

Fold 1


C:\Users\khand\anaconda3\envs\mne\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 15s 55ms/step - accuracy: 0.8632 - loss: 0.3581 - val_accuracy: 0.9133 - val_loss: 0.2312
Epoch 2/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 49ms/step - accuracy: 0.9436 - loss: 0.1597 - val_accuracy: 0.9367 - val_loss: 0.1784
Epoch 3/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 50ms/step - accuracy: 0.9615 - loss: 0.1111 - val_accuracy: 0.9619 - val_loss: 0.1054
Epoch 4/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 49ms/step - accuracy: 0.9673 - loss: 0.0937 - val_accuracy: 0.9656 - val_loss: 0.0950
Epoch 5/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 50ms/step - accuracy: 0.9747 - loss: 0.0752 - val_accuracy: 0.9693 - val_loss: 0.0812
Epoch 6/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 50ms/step - accuracy: 0.9758 - loss: 0.0680 - val_accuracy: 0.9716 - val_loss: 0.0797
Epoch 7/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 48ms/step - accuracy: 0.9810 - loss: 0.0575 - val_accuracy: 0.9716 - val_loss: 0.0814
Epoch 8/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 49ms/step - accuracy: 0.9834 - loss: 0.0464 -

C:\Users\khand\anaconda3\envs\mne\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 16s 57ms/step - accuracy: 0.8488 - loss: 0.3669 - val_accuracy: 0.9193 - val_loss: 0.2161
Epoch 2/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - accuracy: 0.9426 - loss: 0.1546 - val_accuracy: 0.9611 - val_loss: 0.1217
Epoch 3/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - accuracy: 0.9623 - loss: 0.1074 - val_accuracy: 0.9685 - val_loss: 0.0966
Epoch 4/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - accuracy: 0.9698 - loss: 0.0832 - val_accuracy: 0.9711 - val_loss: 0.0869
Epoch 5/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - accuracy: 0.9762 - loss: 0.0687 - val_accuracy: 0.9714 - val_loss: 0.0829
Epoch 6/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - accuracy: 0.9770 - loss: 0.0627 - val_accuracy: 0.9732 - val_loss: 0.0819
Epoch 7/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - accuracy: 0.9810 - loss: 0.0539 - val_accuracy: 0.9758 - val_loss: 0.0718
Epoch 8/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - accuracy: 0.9834 - loss: 0.0485 -

...

Epoch 1/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 18s 62ms/step - accuracy: 0.8489 - loss: 0.3657 - val_accuracy: 0.9277 - val_loss: 0.1963
Epoch 2/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - accuracy: 0.9417 - loss: 0.1599 - val_accuracy: 0.9551 - val_loss: 0.1232
Epoch 3/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 54ms/step - accuracy: 0.9602 - loss: 0.1087 - val_accuracy: 0.9574 - val_loss: 0.1116
Epoch 4/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 54ms/step - accuracy: 0.9673 - loss: 0.0905 - val_accuracy: 0.9632 - val_loss: 0.1016
Epoch 5/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - accuracy: 0.9753 - loss: 0.0703 - val_accuracy: 0.9727 - val_loss: 0.0738
Epoch 6/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 53ms/step - accuracy: 0.9790 - loss: 0.0616 - val_accuracy: 0.9732 - val_loss: 0.0815
Epoch 7/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 53ms/step - accuracy: 0.9821 - loss: 0.0516 - val_accuracy: 0.9756 - val_loss: 0.0771
Epoch 8/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - accuracy: 0.9832 - loss: 0.0483 -

C:\Users\khand\anaconda3\envs\mne\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 21s 75ms/step - accuracy: 0.8485 - loss: 0.3648 - val_accuracy: 0.9332 - val_loss: 0.1854
Epoch 2/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 9s 68ms/step - accuracy: 0.9432 - loss: 0.1506 - val_accuracy: 0.9619 - val_loss: 0.1137
Epoch 3/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 9s 67ms/step - accuracy: 0.9608 - loss: 0.1090 - val_accuracy: 0.9648 - val_loss: 0.0989
Epoch 4/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 9s 68ms/step - accuracy: 0.9694 - loss: 0.0871 - val_accuracy: 0.9732 - val_loss: 0.0791
Epoch 5/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 9s 65ms/step - accuracy: 0.9766 - loss: 0.0662 - val_accuracy: 0.9711 - val_loss: 0.0878
Epoch 6/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 9s 68ms/step - accuracy: 0.9780 - loss: 0.0618 - val_accuracy: 0.9758 - val_loss: 0.0712
Epoch 7/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 9s 65ms/step - accuracy: 0.9816 - loss: 0.0511 - val_accuracy: 0.9740 - val_loss: 0.0756
Epoch 8/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 9s 66ms/step - accuracy: 0.9828 - loss: 0.0498 -

C:\Users\khand\anaconda3\envs\mne\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 19s 69ms/step - accuracy: 0.8556 - loss: 0.3561 - val_accuracy: 0.9387 - val_loss: 0.1949
Epoch 2/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 8s 62ms/step - accuracy: 0.9407 - loss: 0.1556 - val_accuracy: 0.9587 - val_loss: 0.1404
Epoch 3/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 8s 61ms/step - accuracy: 0.9612 - loss: 0.1100 - val_accuracy: 0.9648 - val_loss: 0.1063
Epoch 4/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 8s 62ms/step - accuracy: 0.9681 - loss: 0.0875 - val_accuracy: 0.9690 - val_loss: 0.0873
Epoch 5/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 8s 61ms/step - accuracy: 0.9727 - loss: 0.0730 - val_accuracy: 0.9753 - val_loss: 0.0716
Epoch 6/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 8s 62ms/step - accuracy: 0.9797 - loss: 0.0566 - val_accuracy: 0.9782 - val_loss: 0.0658
Epoch 7/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 8s 59ms/step - accuracy: 0.9818 - loss: 0.0524 - val_accuracy: 0.9777 - val_loss: 0.0634
Epoch 8/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 8s 62ms/step - accuracy: 0.9816 - loss: 0.0513 -

In [13]:
import pandas as pd
metrics_df = pd.DataFrame(metrics_list)

In [14]:
metrics_df.round(4)

,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,ROC_AUC,Kappa,MCC
0,0.9866,0.9871,0.9885,0.9878,0.9864,0.9842,0.9885,0.9729,0.9729
1,0.9884,0.9830,0.9962,0.9896,0.9876,0.9790,0.9962,0.9766,0.9767
2,0.9903,0.9881,0.9943,0.9912,0.9898,0.9854,0.9943,0.9803,0.9804
3,0.9884,0.9867,0.9924,0.9895,0.9880,0.9837,0.9924,0.9766,0.9766
4,0.9855,0.9825,0.9914,0.9869,0.9849,0.9784,0.9914,0.9708,0.9708
5,0.9908,0.9886,0.9947,0.9917,0.9904,0.9860,0.9947,0.9814,0.9814
6,0.9926,0.9910,0.9957,0.9933,0.9923,0.9889,0.9957,0.9851,0.9851
7,0.9876,0.9853,0.9924,0.9888,0.9871,0.9819,0.9924,0.9750,0.9751
8,0.9874,0.9848,0.9924,0.9886,0.9868,0.9813,0.9924,0.9745,0.9745
9,0.9890,0.9872,0.9928,0.9900,0.9885,0.9842,0.9928,0.9777,0.9777


In [15]:
metrics_df.round(4).to_csv('COMBINED_TASK_CNN_LSTM.csv', index = False)

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
result = pd.read_csv("COMBINED_TASK_CNN_LSTM.csv")
result

,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,ROC_AUC,Kappa,MCC
0,0.9866,0.9871,0.9885,0.9878,0.9864,0.9842,0.9885,0.9729,0.9729
1,0.9884,0.9830,0.9962,0.9896,0.9876,0.9790,0.9962,0.9766,0.9767
2,0.9903,0.9881,0.9943,0.9912,0.9898,0.9854,0.9943,0.9803,0.9804
3,0.9884,0.9867,0.9924,0.9895,0.9880,0.9837,0.9924,0.9766,0.9766
4,0.9855,0.9825,0.9914,0.9869,0.9849,0.9784,0.9914,0.9708,0.9708
5,0.9908,0.9886,0.9947,0.9917,0.9904,0.9860,0.9947,0.9814,0.9814
6,0.9926,0.9910,0.9957,0.9933,0.9923,0.9889,0.9957,0.9851,0.9851
7,0.9876,0.9853,0.9924,0.9888,0.9871,0.9819,0.9924,0.9750,0.9751
8,0.9874,0.9848,0.9924,0.9886,0.9868,0.9813,0.9924,0.9745,0.9745
9,0.9890,0.9872,0.9928,0.9900,0.9885,0.9842,0.9928,0.9777,0.9777


In [18]:
(result.mean()*100).round(2)

Accuracy       98.87
Precision      98.64
Recall         99.31
F1             98.97
Sensitivity    98.82
Specificity    98.33
ROC_AUC        99.31
Kappa          97.71
MCC            97.71
dtype: float64

In [19]:
# import csv
# for i in range (1, 11):
#     df = pd.read_csv(f'epoch_{i}.csv')
#     [col] = df.columns
#     split_data = df[col].str.split(';')
#     csv_file = (f'epoch_{i}.csv')
#     # Prepare CSV file and write headers
#     with open(csv_file, mode='w', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow(['Epoch', 'Training Accuracy', 'Training Loss', 'Validation Accuracy', 'Validation Loss'])
#     for j in range (0, 200):
#         with open(csv_file, mode='a', newline='') as file:
#             writer = csv.writer(file)
#             writer.writerow([split_data[j][0], split_data[j][1], split_data[j][2], split_data[j][3], split_data[j][4]])